# Adult Census Income Prediction

## Project Goal

The goal here is to predict whether someone makes more than $50,000/year using census data.  
this notebook walks through the whole pipeline: loading, cleaning, exploring, feature engineering, splitting, scaling, modeling, and evaluation.  
we use the custom courselib library together with newly created custom functions and try to show clear visuals and explanations at each step.

## Pipeline Overview

1.  **Load & Clean Data**
2.  **Exploratory Data Analysis (EDA)**
3.  **Feature Engineering & Preprocessing**
    -   3.1 Category Merging
    -   3.2 Encode categorical variables (target, ordinal, one-hot, frequency)
    -   3.3 Feature Correlations
    -   3.4 Splitting the Data
    -   3.5 Scale Numerical Features
4.  **Model Training**
    -   4.1 Base model
    -   4.2 
    -   4.3 
5.  **Hyperparameter Tuning**
6.  **Model Evaluation & Comparison**

## 1 Load & Clean Data

In [ ]:
# Enable autoreloading of imported modules
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Add the repo root to access the courselib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
courselib_path = os.path.join(repo_root, "AppliedML", "courselib")
if courselib_path not in sys.path:
    sys.path.insert(0, courselib_path)
    print(f"{courselib_path} added to sys.path.")
else:
    print("Courselib path already in sys.path.")

As this project is supposed to integrate well with the courselib, we have downloaded the current GitHub Repo up to week 11 and our code will be integrated within courselib libraries.

In [ ]:
from utils.loaders import load_uciadult

# ensure the data directory exists / else create it
os.makedirs('data', exist_ok=True)

# get the data
df = load_uciadult()

In [ ]:
# check for missing values
df.isnull().sum()

1.  **Handle Duplicates**: We remove any duplicate rows from the dataset.
2.  **Handle Missing Values**: Instead of dropping rows with missing values, we treat them as the separate category  `Missing` in categorical columns. This allows us to preserve potentially useful information — for example, individuals with unknown workclass or occupation might share certain income patterns. This approach is especially appropriate for categorical variables, where missingness itself may contain information. Fortunately, this data set does not contain other missing data.


In [ ]:
from utils.preprocessing import preprocess_data

# preprocessing of  the data
df = preprocess_data(df)

In [ ]:
# first glimpse into the data set
df.head()

# 2. Exploratory Data Analysis (EDA)

In [ ]:
# basic info about the data set 
df.info()

In [ ]:
# summary stats about the variables
df.describe()

In [ ]:
# boxplots of the numerical cols 
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
if 'income' in numerical_cols:
    numerical_cols.remove('income')

# grid of box plots
fig, axes = plt.subplots(nrows=1, ncols=len(numerical_cols), figsize=(20, 6))
fig.suptitle('Box Plots of Numerical Variables', fontsize=16)

for i, col in enumerate(numerical_cols):
    ax = axes[i]
    ax.boxplot(df[col], patch_artist=True, boxprops=dict(facecolor='#00246B'))
    ax.set_title(col, fontsize=12)
    ax.set_ylabel('Value')
    ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

- High Skewness: `fnlwgt`, `capital-gain`, and `capital-loss` are heavily right-skewed. For `capital-gain`, and `capital-loss`, the majority of values are zero, with a few very large outliers. This indicates that these events are rare but have a high magnitude when they occur. `fnlwgt`is dropped as that variable is difficult to interpret. (It indicates how many people in the population a particular record represents and is therefore not an individual attribute and also highly skewed with high variance.)
- Moderate Outliers: age and hours-per-week also show outliers but have more symmetric distributions.

In [ ]:
from utils.preprocessing import log_transform

# skewed columns and apply log transformation to have them on a more similar scale
skewed_cols = ['capital-gain', 'capital-loss']
df = log_transform(df, skewed_cols)
df.drop('fnlwgt', axis=1, inplace=True) # fnlwgt is difficult to interpret

The following chart shows a class imbalance in the data: approximately 76% of individuals earn <=50K (low-earners = 0) while only 24% earn >50K (high-earners = 1). A model trained on this data might become biased towards predicting the majority class. Standard accuracy can be a misleading metric; a naive model that always predicts <=50K would be ~76% accurate but completely useless.

In [ ]:
# distribution of the target variable
plt.figure(figsize=(7, 5))
bars = df['income'].value_counts().sort_index().plot(
    kind='bar',
    color=['#00246B', '#CADCFC'],
)
plt.title('Class Balance of Income', fontsize=16)
plt.xlabel('Income (0 = <=50K, 1 = >50K)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([0, 1], ['<=50K', '>50K'], rotation=0, fontsize=11)
plt.yticks(fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
counts = df['income'].value_counts().sort_index()
labels = ['<=50K', '>50K']
for i, count in enumerate(counts):
    pct = count / counts.sum() * 100
    plt.text(i, count + 200, f'{pct:.1f}%', ha='center', fontsize=11)
plt.show()

The age distribution shows that individuals earning over $50K are, on average, older than those earning less. The high-income group's distribution peaks between the late 30s and early 50s, suggesting that income potential increases with age and experience before declining.

In [ ]:
# relationship between income and age
plt.figure(figsize=(12, 7))

# separate data for the two income categories
over50k = df[df['income'] == 0]['age']
below50k = df[df['income'] == 1]['age']

# side-by-side histograms
plt.hist([over50k, below50k], bins=20, color=['#00246B', '#CADCFC'], label=['<=50K', '>50K'])
plt.title('Age Distribution by Income Level', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Income')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

There seems to exist a strong, positive correlation between education and income. The likelihood of earning over $50K increases directly with the level of education, which seems economically reasonable. This clear ordering suggests the use of ordinal encoding for this feature.

In [ ]:
education_income_pct = df.groupby('education', observed=False)['income'].mean().sort_values() * 100

# plot
plt.figure(figsize=(12, 8))
bars = education_income_pct.plot(kind='barh', color='#00246B')
plt.title('Percentage of Individuals Earning >$50K by Education Level', fontsize=16)
plt.xlabel('Percentage (%)', fontsize=12)
plt.ylabel('Education Level', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.grid(axis='x', linestyle='--', alpha=0.7)

# percentage labels
for bar in bars.patches:
    plt.text(bar.get_width() + 0.1,
             bar.get_y() + bar.get_height() / 2,
             f'{bar.get_width():.1f}%',
             va='center',
             ha='left',
             fontsize=10)

plt.tight_layout()
plt.show()

This chart shows the average hours worked per week for each workclass, with bar color indicating the proportion of high-income earners. `Self-emp-inc` stands out with both the highest average hours and the greatest income potential (55.7%). Conversely, categories like `Without-pay` and `Never-worked` show zero high-income potential. The `Missing` category also has a low income potential (10.4%), suggesting that the reason for the missing data might be correlated with lower-paying or non-standard work situations.

In [ ]:
workclass_stats = df.groupby('workclass', observed=False).agg(
    mean_hours=('hours-per-week', 'mean'),
    pct_high_income=('income', 'mean')
)
overall_stats = pd.DataFrame({
    'mean_hours': [df['hours-per-week'].mean()],
    'pct_high_income': [df['income'].mean()]
}, index=['Overall Average'])

# merge workclass stats with overall stats
combined_stats = pd.concat([workclass_stats, overall_stats]).sort_values('mean_hours', ascending=False)

# plot and colormap
fig, ax = plt.subplots(figsize=(14, 8))
cmap = plt.get_cmap('Blues')
norm = plt.Normalize(vmin=combined_stats['pct_high_income'].min(), vmax=combined_stats['pct_high_income'].max())
colors = cmap(norm(combined_stats['pct_high_income'].values))
bars = ax.bar(combined_stats.index, combined_stats['mean_hours'], color=colors)

# color bar for the mean proportion of high-earners
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, pad=0.05)
cbar.set_label('Proportion Earning >$50K', fontsize=12)

# text labels for the proportions
labels = [f'{pct*100:.1f}%' for pct in combined_stats['pct_high_income']]
ax.bar_label(bars, labels=labels, padding=3, fontsize=10, color='black')
ax.set_title('Mean Hours Worked by Workclass (vs. Overall Average)', fontsize=16)
ax.set_xlabel('Work Class / Category', fontsize=12)
ax.set_ylabel('Mean Hours per Week', fontsize=12)
ax.set_ylim(0, combined_stats['mean_hours'].max() * 1.1)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# 3. Feature Engineering


In [ ]:
# counts and proportions of every categorical col
categorical_cols_merged = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols_merged:
    print(f"--- Feature: {col} ---")
    
    # summary of each group within a col
    summary_df = pd.DataFrame({
        'Count': df[col].value_counts(),
        'Proportion (%)': (df[col].value_counts(normalize=True) * 100).round(2)
    })
    print(summary_df)
    print("_"*40 + "\n")

### 3.1 Category Merging

Before encoding, we will merge some categories to reduce dimensionality and group similar items to simplify features and help the model generalize better. Thus several possible categorical variables are analyzed and discussed.


In [ ]:
# to modify the dataset
df = df.copy()

In [ ]:
# binary feature for US vs. rest of the world as the dataset is mostly US-related
df['is_from_us'] = (df['native-country'] == 'United-States').astype(int)
df.drop('native-country', axis=1, inplace=True)
print("Value counts for 'is_from_us':\n")
print(df['is_from_us'].value_counts())

In [ ]:
# pie chart for US vs. Non-US
us_vs_nonus = df.groupby('is_from_us')['income'].mean()
labels = ['Non-US', 'US']
plt.figure(figsize=(5, 5))
plt.pie(us_vs_nonus, labels=labels, autopct='%1.1f%%', colors=['#CADCFC', '#00246B'], startangle=90)
plt.title('Proportion Earning >$50K: US vs. Non-US')
plt.show()

In [ ]:
# mean income by marital status

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Count of each marital status
marital_counts = df['marital-status'].value_counts().sort_values(ascending=False)
marital_counts.plot(kind='bar', color='#00246B', ax=axes[0])
axes[0].set_title('Distribution of Marital Status')
axes[0].set_xlabel('Marital Status')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=30)

# Mean income by marital status
marital_income = df.groupby('marital-status', observed=False)['income'].mean().sort_values(ascending=False) * 100
marital_income.plot(kind='bar', color='#CADCFC', ax=axes[1])
axes[1].set_title('Mean Income >$50K by Marital Status')
axes[1].set_xlabel('Marital Status')
axes[1].set_ylabel('Proportion >$50K (%)')
axes[1].tick_params(axis='x', rotation=30)
for i, v in enumerate(marital_income):
    axes[1].text(i, v + 0.5, f"{v:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

The plot suggests that creating a binary feature for married vs. non-married, although we will treat the group Married-spouse-absent, i.e. legally married but not living with spouse (spouse is absent, e.g., separated but not divorced) as non-married.

In [ ]:
married_statuses = ['Married-civ-spouse', 'Married-AF-spouse']
df['is_married'] = df['marital-status'].isin(married_statuses).astype(int)
df.drop('marital-status', axis=1, inplace=True)
print("\nValue counts for the new 'is_married' feature:\n")
print(df['is_married'].value_counts())

In [ ]:
# merge husband and wife into spouse in relationship variable to further reduce cardinality for onehot encoding
df['relationship'] = df['relationship'].astype(str).replace({'Husband': 'Spouse', 'Wife': 'Spouse'}).astype('category')

# merge without-pay, never-worked categories into other to further reduce cardinality for onehot encoding
df['workclass'] = df['workclass'].astype(str).replace(['Without-pay', 'Never-worked'], 'Other').astype('category')

In [ ]:
# race distribution and mean income by race
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# count of each race
race_counts = df['race'].value_counts().sort_values(ascending=False)
race_counts.plot(kind='bar', color='#00246B', ax=axes[0])
axes[0].set_title('Distribution of Race')
axes[0].set_xlabel('Race')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=30)

# mean income by race
race_income = df.groupby('race', observed=False)['income'].mean().sort_values(ascending=False) * 100
race_income.plot(kind='bar', color='#CADCFC', ax=axes[1])
axes[1].set_title('Mean Income >$50K by Race')
axes[1].set_xlabel('Race')
axes[1].set_ylabel('Proportion >$50K (%)')
axes[1].tick_params(axis='x', rotation=30)
for i, v in enumerate(race_income):
    axes[1].text(i, v + 0.2, f"{v:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# merge Amer-Indian-Eskimo and Other categories into other to further reduce cardinality for onehot encoding
df['race'] = df['race'].astype(str).replace({'Amer-Indian-Eskimo': 'Other'}).astype('category')

The merging of categories in features like race, relationship, and workclass leads to only a small reduction in dimensionality, but is justified because the combined groups are both small and statistically similar in terms of income distribution. This reduces noise and the risk of overfitting, while preserving the main socioeconomic patterns in the data. By merging only where appropriate, we maintain interpretability and model performance without unnecessary simplification.

In [ ]:
# number of distinct values in our categorical cols -> what feature encoding strat to use
categorical_vars = ['workclass', 'relationship', 'race', 'sex', 'education', 'occupation']

for var in categorical_vars:
    if var in df.columns:
        unique_vals = df[var].unique()
        print(f"{var}: {len(unique_vals)} unique values:\n {list(unique_vals)}\n")

### 3.2 Data Splitting

To prevent data leakage, we must split the data into training and testing sets *before* any encoding or scaling. Previously, this mistake lead to artificially good performance on the validation set. All fitting steps (like for encoders and scalers) will be performed on the training set only and then applied to both the training and test sets.

In [ ]:
from utils.splits import train_test_split

# returns the full dataset (X, y)
df, train_df, test_df = train_test_split(
    df, 
    training_data_fraction=0.8, 
    class_column_name='income',
    shuffle=True, 
    return_numpy=False # dataframes instead of numpy arrays
)

print("Train df shape:", train_df.shape)
print("Test df shape:", test_df.shape)

### 3.3 Feature Encoding strategies

Feature encoding transforms categorical variables into numerical values for machine learning models. We will apply one-hot and ordinal encoding first, followed by a careful application of target encoding to prevent data leakage.

Brief explanation of all the strategies for encoding:

#### 1. One-Hot Encoding

- **Def.:** For a feature with $k$ categories, create $k-1$ binary columns:
  $$
  \text{OneHot}(x = c_j) = [0, \ldots, 1, \ldots, 0] \in \mathbb{R}^{k-1}
  $$
  where the $j$-th position is $1$ if $x = c_j$, else $0$.
- **Example:** If `race` has categories `[White, Black, Asian]` and $x = \text{Black}$, then $\text{OneHot}(x) = [1, 0]$.
- **When/Why:** Use for nominal (unordered) features to avoid implying order. We create $k-1$ columns instead of $k$ to avoid **perfect multicollinearity**, where one feature can be perfectly predicted by the others. This is crucial for linear models. The dropped category  (the most frequent one) becomes the baseline reference.

---

#### 2. Ordinal Encoding

- **Def.:** Map ordered categories to integers:
  $$
  \text{Ordinal}(x = c_j) = j
  $$
  where $j$ is the position in the predefined order.
- **Example:** If `education` order is `[HS-grad, Bachelors, Masters]$ and $x = \text{Bachelors}$, then $\text{Ordinal}(x) = 1$.
- **When/Why:** Use for ordinal features where order matters.

---

#### 3. Target Encoding

- **Def.:** Replace each category with the mean target value:
  $$
  \text{TargetEnc}(x = c_j) = \mathbb{E}[y \mid x = c_j]
  $$
- **Example:** If mean income for `occupation = Tech` is $0.32$, then $\text{TargetEnc}(\text{Tech}) = 0.32$.
- **When/Why:** Use for high-cardinality features; captures target signal but risk of leakage (use cross-validation).

---

#### 4. Frequency Encoding

- **Def.:** Replace each category with its frequency:
  $$
  \text{FreqEnc}(x = c_j) = \frac{\text{count}(x = c_j)}{N}
  $$
  where $N$ is the total number of samples.
- **Example:** If `country = US` appears in $70\%$ of rows, then $\text{FreqEnc}(\text{US}) = 0.7$.
- **When/Why:** Use for high-cardinality features; simple, no leakage.

In [ ]:
from utils.preprocessing import encode_and_align_features

# encoding strategies for each column we need to encode
base_encoding = {'one-hot': { 'relationship' : 'Spouse',
                'race' : 'White', # second entry is dropped to avoid multicollinearity
                'sex' : 'Male'},
    'ordinal': {'education': ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad',
                               'Some-college', 'Assoc-voc', 'Assoc-acdm', 'Bachelors', 'Masters', 'Prof-school',
                               'Doctorate']},
    'target': {'occupation': 'income'},  # Format: {feature_to_encode: target_column}
    'frequency': ['workclass']
}

train_df_encoded, test_df_encoded, fit_params = encode_and_align_features(
    train_df, test_df, base_encoding, drop_cols=['education-num'])

### 3.4 Correlation of Features with the target variable

In [ ]:
# bar chart of correlations with income
correlation_matrix_encoded = train_df_encoded.corr()
income_correlations = correlation_matrix_encoded['income'].drop('income').sort_values()
plt.figure(figsize=(8, 5))
income_correlations.plot(kind='barh', color='#00246B')
plt.title('Correlation of Features with Income in Training Data', fontsize=16)
plt.xlabel('Pearson Correlation Coefficient', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Interpretation:

*   **Positive Correlation (Features on the right):** Features with a positive bar indicate that as the feature's value increases, the likelihood of having an income `>50K` also increases.
*   **Negative Correlation (Features on the left):** Features with a negative bar indicate that as the feature's value increases, the likelihood of having an income `>50K` decreases.

*   **Strongest Predictors:** `is_married`, `education` (ordinal), `age`, `hours-per-week`, and `capital-gain` show the strongest positive correlations. This aligns with our EDA findings: being married, having higher education, being older, and working more hours are all strongly associated with higher income. The target-encoded `occupation` feature is also a very strong predictor by design.

*   **Negative Predictors:** The strongest negative correlations come from one-hot encoded features like `relationship_Own-child` and `relationship_Not-in-family`. This indicates that individuals in these relationship categories are significantly less likely to have high income.


The dropped variables are the silent baseline against which all the other one-hot encoded columns from that same feature are measured. Their relationships are not directly visible in the matrix, but are implicitly defined by the relationships of the columns that are present. The relationship of the dropped `race_White` category is the inverse of the combined effect of the other categories. Because `race_White` is the most frequent category, the other, smaller race categories being negatively correlated with income implies that the large, baseline `race_White` category must have a correlation that is neutral or slightly positive to balance things out.

### 3.5 Data Splitting into features and target

In [ ]:
from utils.splits import split_features_and_target
X_train, y_train, X_test, y_test = split_features_and_target(train_df_encoded, test_df_encoded)

### 3.6 Feature Scaling

Before training our models, it's important to scale our numerical features. Of course, it makes no sense to also scale the freshly encoded categorical data, such as the ordinally encoded `education` variable as we would destroy its ordinal structure immediately again.  Models like Logistic Regression are sensitive to the scale of the input data, and scaling can lead to faster convergence and better performance. We will use standardization, where each feature is scaled to have a mean of 0 and a standard deviation of 1.

We will fit the scaler on the training data and use the same scaler to transform the test data. This prevents any information from the test set from leaking into our training process.


In [ ]:
# all columns in the data set
train_df_encoded.columns

In [ ]:
from utils.normalization import scale_numerical_features

numerical_cols = [
    'capital-gain', 'capital-loss', 'age', 'hours-per-week',
    'education_ordinal', 'occupation_target', 'workclass_freq'
]

# scale features via normalization
X_train_scaled, X_test_scaled = scale_numerical_features(
    X_train, X_test, train_df_encoded, numerical_cols
)

## 4. Model Training (Logistic Regression)

Predict whether an individual's income exceeds $50,000 per year.

### Models Included:
- **Logistic Regression**: A logistic Regression model for binary classification, optimized using gradient descent.

### Evaluation Metrics:
- **Accuracy**: The proportion of total correct predictions.
  $$ \text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN} $$
- **Precision**: The accuracy of positive predictions.
  $$ \text{Precision} = \frac{TP}{TP + FP} $$
- **Recall (Sensitivity)**: The ability of the model to find all the positive samples.
  $$ \text{Recall} = \frac{TP}{TP + FN} $$
- **Training Time**: time taken to train the model to evaluate efficiency.
- **Confusion Matrix**: A table visualizing the counts of TP, TN, FP, and FN.

In [ ]:
from models.glm import LogisticRegression
from optimizers import GDOptimizer
from utils.training import ModelTraining
from utils.metrics import accuracy, precision_score, recall_score


# model params
n_features = X_train_scaled.shape[1]
w_init = np.zeros(n_features)
b_init = 0.0
optimizer = GDOptimizer(learning_rate=0.01)

# to save the results from training
results = []

# metrics
metrics_dict = {
    "Accuracy": lambda y_true, y_pred: accuracy(y_pred, y_true, one_hot_encoded_labels=False),
    "Precision": precision_score,
    "Recall": recall_score,
}

# model configurations
model_configs = [
    {
        'model_class': LogisticRegression,
        'model_name': 'Logistic Regression',
        'init_params': {
            'w': w_init.copy(), 
            'b': b_init, 
            'optimizer': GDOptimizer(learning_rate=0.01),
            'penalty': 'none', # 'none', 'ridge', 'lasso'
            'lam': 0.0
        },
        'fit_params': {'num_epochs': 20, 'batch_size': 32}
    }
]

# specify the models to train
models_to_train = []
for config in model_configs:
    model_to_train = ModelTraining(
        model_class=config['model_class'],
        model_name=config['model_name'],
        init_params=config['init_params'],
        metrics_dict=metrics_dict,
        **config['fit_params']
    )
    models_to_train.append(model_to_train)

### 4.1 Base model with detailed evaluation

First train only one model with logistic regression and without cross-validation to analyse that in detail. Then with 5-fold cv for more accurate results.

In [ ]:
from utils.metrics import plot_training_history

# training process of one model without cv
for model_to_train in models_to_train:
    
    # train the model using the params defined in the dict
    print(100*'=')
    print(f"Model: {model_to_train.model_name}")
    model_to_train.train(X_train_scaled, y_train, X_test_scaled, y_test)
    
    # plot training metrics (accuracy and loss for train/test)
    if model_to_train.history:
        plot_training_history(model_to_train.history, model_to_train.model_name)
        print(100*'=')

    # eval on the test set and store results
    performance = model_to_train.evaluate(X_test_scaled, y_test)
    results.append(performance)
    print("="*60)

In [ ]:
# results for the simple training run
results_df = pd.DataFrame(results)
display(results_df)

### 4.2 Compare different regularized versions of Logistic Regression

In [ ]:
from utils.training import ModelTraining

# model configurations
logreg_configs = [
    {
        'model_class': LogisticRegression,
        'model_name': 'Logistic Regression (Base)',
        'init_params': {
            'w': w_init.copy(), 
            'b': b_init, 
            'optimizer': GDOptimizer(learning_rate=0.01),
            'penalty': 'none',
            'lam': 0.0
        },
        'fit_params': {'num_epochs': 20, 'batch_size': 32}
    },
    {
        'model_class': LogisticRegression,
        'model_name': 'Logistic Regression (Weak Ridge)',
        'init_params': {
            'w': w_init.copy(), 
            'b': b_init, 
            'optimizer': GDOptimizer(learning_rate=0.01),
            'penalty': 'ridge',
            'lam': 0.1
        },
        'fit_params': {'num_epochs': 20, 'batch_size': 32}
    },   
    {
        'model_class': LogisticRegression,
        'model_name': 'Logistic Regression (Strong Ridge)',
        'init_params': {
            'w': w_init.copy(), 
            'b': b_init, 
            'optimizer': GDOptimizer(learning_rate=0.01),
            'penalty': 'lasso',
            'lam': 1
        },
        'fit_params': {'num_epochs': 20, 'batch_size': 32}
    },
    {
        'model_class': LogisticRegression,
        'model_name': 'Logistic Regression (Lasso)',
        'init_params': {
            'w': w_init.copy(), 
            'b': b_init, 
            'optimizer': GDOptimizer(learning_rate=0.01),
            'penalty': 'lasso',
            'lam': 0.1
        },
        'fit_params': {'num_epochs': 20, 'batch_size': 32}
    }
]

# specify the logreg models to train
logreg_models_to_train = []
for logreg_config in logreg_configs:
    logreg_model_to_train = ModelTraining(
        model_class=logreg_config['model_class'],
        model_name=logreg_config['model_name'],
        init_params=logreg_config['init_params'],
        metrics_dict=metrics_dict,
        plot_cm=False,
        **logreg_config['fit_params']
    )
    logreg_models_to_train.append(logreg_model_to_train)

In [ ]:
# train the models via 5-fold cv now
all_cv_results = []
for logreg_model_to_train in logreg_models_to_train:
    # k-fold cross-validation for current model
    cv_result = logreg_model_to_train.cross_validate(X_train_scaled, y_train, k_folds=5)
    all_cv_results.append(cv_result)
    print("="*60)

In [ ]:
# final summary of the cv pf all models tested
cv_results_df = pd.DataFrame(all_cv_results)
display(cv_results_df)

#### Why all those metrics
- Accuracy can be misleading: a model could achieve ~76% accuracy by simply always predicting the majority class (<=50K), making it useless.
- Precision: "Of all the people we predicted would have high income, how many actually did?" Important if the cost of a false positive is high.
- Recall: "Of all the people who actually have high income, how many did we find?" Important if the cost of a false negative is high.
- Confusion Matrix: provides the raw numbers (TP, TN, FP, FN) from which all the other metrics are calculated

## 6 Hyperparameter tuning of those models

In [ ]:
def gridsearch_logistic_regression(X_train, y_train, X_test, y_test, param_grid, cv_folds=None):
    """
    Performs a grid search for Logistic Regression.
    """
    results = []
    
    # parameter grid
    learning_rates = param_grid.get('lr', [0.01])
    epochs_list = param_grid.get('num_epochs', [20])
    penalties = param_grid.get('penalty', ['none'])
    lambdas = param_grid.get('lam', [0.0])

    for lr in learning_rates:
        for num_epochs in epochs_list:
            for penalty in penalties:
                for lam in lambdas:
                    # Skip non-reg if lam is not 0
                    if penalty == 'none' and lam != 0.0:
                        continue
                    # Skip reg if lam is 0
                    if penalty != 'none' and lam == 0.0:
                        continue
                        
                    print(f"Training with lr={lr}, epochs={num_epochs}, penalty='{penalty}', lambda={lam}")

                    # init model with hyperparameters
                    optimizer = GDOptimizer(learning_rate=lr)
                    model = LogisticRegression(
                        w=np.zeros(X_train.shape[1]),
                        b=0.0,
                        optimizer=optimizer,
                        penalty=penalty,
                        lam=lam
                    )
                    
                    # ModelTraining wrapper for training/evaluation
                    trainer = ModelTraining(
                        model_class=LogisticRegression,
                        model_name="Tuning",
                        init_params={'w': model.w, 'b': model.b, 'optimizer': optimizer, 'penalty': penalty, 'lam': lam},
                        metrics_dict=metrics_dict,
                        num_epochs=num_epochs,
                        batch_size=32,
                        plot_cm=False
                    )
                    if cv_folds is not None and cv_folds > 1:
                        # cross-validation mode
                        cv_result = trainer.cross_validate(X_train, y_train, k_folds=cv_folds)
                        # add hyperparams to cv_results
                        cv_result.update({
                            "Learning Rate": lr,
                            "Epochs": num_epochs,
                            "Penalty": penalty,
                            "Lambda": lam
                            })
                        results.append(cv_result)
                    else:
                        # normal training - no cv
                        trainer.train(X_train, y_train)
                        performance = trainer.evaluate(X_test, y_test)
                        results.append({
                            "Learning Rate": lr,
                            "Epochs": num_epochs,
                            "Penalty": penalty,
                            "Lambda": lam,
                            "Accuracy": performance['Accuracy'],
                            "Precision": performance['Precision'],
                            "Recall": performance['Recall']
                        })

    return pd.DataFrame(results)


# grid of hyperparameters
param_grid = {
    'lr': [0.01], #, 0.05, 0.1, 0.5],
    'num_epochs': [20], #, 30, 50, 100],
    'penalty': ['none', 'ridge', 'lasso'],
    'lam': [0.0, 0.00001, 0.1], #, 1, 10, 100.0]
}


logreg_tuning_results = gridsearch_logistic_regression(
    X_train_scaled, y_train, X_test_scaled, y_test,
    param_grid,
    cv_folds=5 # optional param as cv takes a lot of time - can be uncommented for faster performance
)

In [ ]:
display(logreg_tuning_results)

### 6.2 Analyse the best performing model

In [ ]:
# find the best model by (mean) accuracy
if "Mean Accuracy" in logreg_tuning_results.columns:
    best_row = logreg_tuning_results.sort_values("Mean Accuracy", ascending=False).iloc[0]
    best_acc = best_row["Mean Accuracy"]
else:
    best_row = logreg_tuning_results.sort_values("Accuracy", ascending=False).iloc[0]
    best_acc = best_row["Accuracy"]

# get the params of the best model
best_penalty = best_row["Penalty"]
best_lam = best_row["Lambda"]
best_lr = best_row["Learning Rate"]
best_epochs = best_row["Epochs"]
print(f"Best model: penalty={best_penalty}, lambda={best_lam}, lr={best_lr}, epochs={best_epochs}")

# retrain on the full training set - no cv here
best_optimizer = GDOptimizer(learning_rate=best_lr)
best_model = LogisticRegression(
    w=np.zeros(X_train_scaled.shape[1]),
    b=0.0,
    optimizer=best_optimizer,
    penalty=best_penalty,
    lam=best_lam
)

In [ ]:
print("Initial weights:", best_model.w)
print("Initial bias:", best_model.b)

In [ ]:
best_trainer = ModelTraining(
    model_class=LogisticRegression,
    model_name="Best Logistic Regression",
    init_params={'w': best_model.w, 'b': best_model.b, 'optimizer': best_optimizer, 'penalty': best_penalty, 'lam': best_lam},
    metrics_dict=metrics_dict,
    num_epochs=int(best_epochs),
    batch_size=32,
    plot_cm=True
)
best_trainer.train(X_train_scaled, y_train, X_test_scaled, y_test)
# plot training metrics (accuracy and loss for train/test)
if model_to_train.history:
    plot_training_history(best_trainer.history, best_trainer.model_name)

In [ ]:
print("Weights after training:", best_trainer.model.w)
print("Bias after training:", best_trainer.model.b)

In [ ]:
# analyze coefficients
coefs = best_trainer.model.w
feature_names = [col for col in train_df_encoded.columns if col != 'income']
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefs})
# 10 most important features
top_pos = coef_df.sort_values(by='coefficient', ascending=False).head(5)
top_neg = coef_df.sort_values(by='coefficient').head(5)
print("Top 5 Positive Coefficients (features that increase prob of >50K):")
print(top_pos)
print("\nTop 5 Negative Coefficients (features that decrease prob of >50K):")
print(top_neg)

In [ ]:
# plot the importance of all features for our best model, sorted by absolute value
coef_df_sorted = coef_df.reindex(coef_df['coefficient'].sort_values(ascending=False).index)
plt.figure(figsize=(10, 6))
plt.barh(coef_df_sorted['feature'], coef_df_sorted['coefficient'], color=['#00246B' if c > 0 else '#CADCFC' for c in coef_df_sorted['coefficient']])
plt.xlabel('Coefficient Value in Logistic Regression')
plt.title('Importance of the Features on the Probability of >$50K Income')
plt.axvline(0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

Intrepreting the negative coefficient on is_from_us

In [ ]:
print(train_df.groupby('is_from_us')['income'].mean())

In [ ]:
# does this stem from multicollinearity?
corr = train_df_encoded.corr()
print(corr['is_from_us'].sort_values(ascending=False))

In [ ]:
# frequency of non-US and income
print(train_df_encoded.groupby('is_from_us')['income'].mean())
print(train_df_encoded['income'].value_counts())
print(train_df_encoded['is_from_us'].value_counts())

In [ ]:
from models.glm import LogisticRegression
from optimizers import GDOptimizer

# small model only on people from the US 
X_us = train_df_encoded[['is_from_us']].values
y = train_df_encoded['income'].values
model = LogisticRegression(w=np.zeros(1), b=0.0, optimizer=GDOptimizer(learning_rate=0.01))
for _ in range(100):
    grads = model.loss_grad(X_us, y)
    model.w -= 0.01 * grads['w']
    model.b -= 0.01 * grads['b']
print("Coefficient for is_from_us (only feature in model):", model.w[0])

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(fit_intercept=True)
model.fit(train_df[['is_from_us']], train_df['income'])
print(model.coef_, model.intercept_)

### Interpreting Odds Ratios for Logistic Regression Coefficients

This shows the **odds ratios** for each feature in the logistic regression model. The odds ratio is calculated as `exp(coefficient)` and represents the multiplicative change in the odds of earning >$50K for a one-unit increase in the feature, holding all other features constant:

- **Odds Ratio > 1:** The feature increases the odds of high income. For example, an odds ratio of 2 means the odds double for each unit increase.
- **Odds Ratio < 1:** The feature decreases the odds of high income. For example, an odds ratio of 0.5 means the odds are halved for each unit increase.
- **Odds Ratio ≈ 1:** The feature has little or no effect on the odds.

#### What is a "unit increase" for each feature?

- **Standardized numerical features**: unit increase means **one standard deviation above the mean** of that feature .
  - For `capital-gain` and `capital-loss`, this is after a `log1p` transformation, so it reflects a standard deviation in the log-transformed value.

- **Binary features**: changing from 0 to 1 (e.g., from not married to married).

- **One-hot encoded features**: unit increase means belonging to that category versus the baseline (reference) category.

- **Ordinally encoded features**: unit increase means moving up by one level in the encoded order, but after standardization, it is also in units of standard deviation.

In [ ]:
# Odds ratios for the features
coef_df["odds_ratio"] = np.exp(coef_df["coefficient"])
print("Top features with odds ratios:")
display(coef_df.loc[coef_df.index][["feature", "coefficient", "odds_ratio"]])